In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Input, GlobalAveragePooling2D
from tensorflow.keras.utils import to_categorical
import os

from tensorflow.keras.optimizers import Adam
os.environ["CUDA_VISIBLE_DEVICES"] = '1'
# 加载 CIFAR-10 数据
import random
import numpy as np
# Load and preprocess CIFAR-10 data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0
y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)


2024-08-13 09:45:54.521841: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


In [2]:
##该模型并没有使用双重对齐，因为我使用的数据错了。

In [3]:

def add_trigger(img):
    trigger_size =4
    img[-trigger_size:, -trigger_size:, 0] =0# 设置为红色
    img[-trigger_size:, -trigger_size:, 1:3] = 0
    return img


# Apply the SIG backdoor attack
# Apply the SIG backdoor attack to the training set

# 选择1%的训练数据进行攻击

target_class = 1  # 目标类别
num_classes = 10  # CIFAR-10 数据集的类别数

# 新的数据集和标签列表
x_val_triggered = []  # 存储触发器转换后的图像
y_val_triggered = []  # 存储更新后的标签

x_val_tigger = x_val.copy()
y_val_tigger = y_val.copy()
y_val_tigger_2 = np.zeros(len(y_val))  # 所有标签初始化为0
# 将标签转换为one-hot编码
y_val_tigger_2 = to_categorical(y_val_tigger_2, num_classes=2)


num_samples = int(0.1 * x_val.shape[0])
indices = np.random.choice(y_val.shape[0], num_samples, replace=False)



# 应用攻击到选中的图像
for i in indices:
    x_val_tigger[i] = add_trigger(x_val_tigger[i])
    y_val_tigger[i] = to_categorical(target_class, num_classes=num_classes)
    y_val_tigger_2[i] = to_categorical(target_class, num_classes = 2 )
        
    
x_test_tigger = x_test.copy()
y_test_tigger = y_test.copy()

for i in range(len(x_test_tigger)):
    x_test_tigger[i] = add_trigger(x_test_tigger[i])
    y_test_tigger[i] = to_categorical(target_class, num_classes=num_classes)
##进行攻击的思路
#1.将数据集分成两部分  8比2吧
#2.每个epcoch后再单独优化一下服务器模型，保持模拟的客户端模型不变，后门数据要多一些
#3.验证数据集结果



    

In [4]:
import tensorflow as tf
from tensorflow.keras import layers, models


def create_client_model(input_shape):
    input_layer = layers.Input(shape=input_shape)
    
    # 第一层卷积
    x = layers.Conv2D(32, 3, strides=1, padding='same')(input_layer)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    
    # 第二层卷积
    x = layers.Conv2D(64, 3, strides=1, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    
    # 新增的第三层卷积
    x = layers.Conv2D(128, 3, strides=1, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    
    model = models.Model(inputs=input_layer, outputs=x)
    return model


In [5]:
from tensorflow.keras import layers, models

def res_block(filters, strides):
    def block(x):
        shortcut = x

        x = layers.Conv2D(filters, 3, padding='same', strides=strides)(x)
        x = layers.BatchNormalization()(x)
        x = layers.Activation('relu')(x)

        x = layers.Conv2D(filters, 3, padding='same')(x)
        x = layers.BatchNormalization()(x)

        # 捷径连接前的调整
        shortcut = layers.Conv2D(filters, 1, strides=strides, padding='same')(shortcut)
        shortcut = layers.BatchNormalization()(shortcut)

        x = layers.add([x, shortcut])
        x = layers.Activation('relu')(x)
        return x
    return block

def create_server_model():
    # 调整输入层的定义以匹配修改后的客户端模型的输出
    input_layer = layers.Input(shape=(32, 32, 128))  # 注意这里的变化

    # 残差块定义保持不变，继续使用提前定义的 res_block
    x = res_block(64, 1)(input_layer)  # 使用第一个残差块
    x = res_block(128, 2)(x)            # 使用第二个残差块
    x = res_block(128, 2)(x)           # 使用第三个残差块
    x = res_block(256, 2)(x)           # 使用第四个残差块

    # 全局平均池化和输出层保持不变
    x = layers.GlobalAveragePooling2D()(x)
    output = layers.Dense(10, activation='softmax')(x)
    
    model = models.Model(inputs=input_layer, outputs=output)
    return model

In [6]:
def create_aux_model():
    # 调整输入层的定义以匹配修改后的客户端模型的输出
    input_layer = layers.Input(shape=(32, 32, 128))  # 注意这里的变化

    # 第一个卷积层和ReLU激活
    x = layers.Conv2D(128, 3, strides=2, padding='same')(input_layer)
    x = layers.ReLU()(x)

    # 第二个卷积层和ReLU激活
    x = layers.Conv2D(128, 3, strides=2, padding='same')(x)
    x = layers.ReLU()(x)

    # 添加残差块
    x = res_block(256, 1)(x)  # 使用第一个残差块
    x = res_block(256, 1)(x) # 使用第二个残差块

    # 第三个卷积层和ReLU激活
    x = layers.Conv2D(256, 3, strides=2, padding='same')(x)
    x = layers.ReLU()(x)

    # 全连接层
    x = layers.Flatten()(x)
    output = layers.Dense(2, activation='softmax')(x)
    
    model = models.Model(inputs=input_layer, outputs=output)
    return model


In [7]:
aux_model = create_aux_model()

2024-08-13 09:46:28.872236: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2024-08-13 09:46:29.019038: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:86:00.0 name: NVIDIA Tesla V100-PCIE-16GB computeCapability: 7.0
coreClock: 1.38GHz coreCount: 80 deviceMemorySize: 15.78GiB deviceMemoryBandwidth: 836.37GiB/s
2024-08-13 09:46:29.019142: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2024-08-13 09:46:29.284515: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcublas.so.10
2024-08-13 09:46:29.368336: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcufft.so.10
2024-08-13 09:46:29.424670: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcu

In [8]:
def create_discrim_model():
    # 调整输入层的定义以匹配修改后的客户端模型的输出
    input_layer = layers.Input(shape=(32, 32, 128))   # 注意这里的变化

    # 第一个卷积层和ReLU激活
    x = layers.Conv2D(128, 3, strides=2, padding='same')(input_layer)
    x = layers.ReLU()(x)

    # 第二个卷积层和ReLU激活
    x = layers.Conv2D(128, 3, strides=2, padding='same')(x)
    x = layers.ReLU()(x)

    # 添加三个残差块
    x = res_block(256, 1)(x)  # 使用第一个残差块
    x = res_block(256, 1)(x)  # 使用第二个残差块
    x = res_block(256, 1)(x)  # 使用第三个残差块

    # 第三个卷积层和ReLU激活
    x = layers.Conv2D(256, 3, strides=2, padding='same')(x)
    x = layers.ReLU()(x)

    # 全连接层，输出单一值
    x = layers.Flatten()(x)
    output = layers.Dense(1, activation='sigmoid')(x)
    
    model = models.Model(inputs=input_layer, outputs=output)
    return model


In [9]:
discrim_model = create_discrim_model()

In [10]:
discrim_model.summary()

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 32, 32, 128) 0                                            
__________________________________________________________________________________________________
conv2d_9 (Conv2D)               (None, 16, 16, 128)  147584      input_2[0][0]                    
__________________________________________________________________________________________________
re_lu_3 (ReLU)                  (None, 16, 16, 128)  0           conv2d_9[0][0]                   
__________________________________________________________________________________________________
conv2d_10 (Conv2D)              (None, 8, 8, 128)    147584      re_lu_3[0][0]                    
_______________________________________________________________________________________

In [11]:
server_model = create_server_model()

In [12]:
# 创建客户端模型
client_model = create_client_model(input_shape=(32, 32, 3))

# 编译客户端模型
client_model.compile(optimizer=Adam(),
                     loss='categorical_crossentropy',
                     metrics=['accuracy'])


In [13]:
# 以下是服务器模型训练的简化示例
server_model.compile(optimizer=Adam(),
                      loss='categorical_crossentropy',
                      metrics=['accuracy'])
discrim_model.compile(optimizer=Adam(),
                      loss='categorical_crossentropy',
                      metrics=['accuracy'])
aux_model.compile(optimizer=Adam(),
                      loss='categorical_crossentropy',
                      metrics=['accuracy'])


In [14]:
##创建三个影子模型，分别是1层卷积，2层卷积核3层卷积


In [15]:
def create_client_model_1(input_shape):
    input_layer = layers.Input(shape=input_shape)
    # 确保输出为128个通道
    x = layers.Conv2D(128, (3, 3), padding='same', activation='relu')(input_layer)
    model = models.Model(inputs=input_layer, outputs=x)
    return model
def create_client_model_2(input_shape):
    input_layer = layers.Input(shape=input_shape)
    x = layers.Conv2D(64, (3, 3), padding='same', activation='relu')(input_layer)
    # 第二层输出调整为128个通道
    x = layers.Conv2D(128, (3, 3), padding='same', activation='relu')(x)
    model = models.Model(inputs=input_layer, outputs=x)
    return model
def create_client_model_3(input_shape):
    input_layer = layers.Input(shape=input_shape)
    x = layers.Conv2D(32, (3, 3), padding='same', activation='relu')(input_layer)
    x = layers.Conv2D(64, (3, 3), padding='same', activation='relu')(x)
    x = layers.Conv2D(128, (3, 3), padding='same')(x)  # 移除这里的激活函数
    x = layers.Activation('relu')(x)  # 显式添加ReLU激活层
    model = models.Model(inputs=input_layer, outputs=x)
    return model

def create_client_model_4(input_shape):
    input_layer = layers.Input(shape=input_shape)
    x = layers.Conv2D(32, (3, 3), padding='same', activation='relu')(input_layer)
    x = layers.Conv2D(64, (3, 3), padding='same', activation='relu')(x)
    x = layers.Conv2D(64, (3, 3), padding='same', activation='relu')(x)
    # 第四层输出调整为128个通道
    x = layers.Conv2D(128, (3, 3), padding='same', activation='relu')(x)
    model = models.Model(inputs=input_layer, outputs=x)
    return model


In [16]:
# Assuming TensorFlow is installed and the functions are defined in your script
input_shape = (32, 32, 3)

client_model_1 = create_client_model(input_shape=(32, 32, 3))

In [17]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [18]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers

# 定义优化器
# 定义第一部分的数据集
def augment(image, label):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_brightness(image, max_delta=0.1)
    return image, label

train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = train_dataset.map(augment).shuffle(len(x_train)).batch(32)

# 定义第二部分的数据集
def preprocess(image, label1, label2):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_brightness(image, max_delta=0.1)
    return image, (label1, label2)

train_dataset_2 = tf.data.Dataset.from_tensor_slices((x_val_tigger, y_val_tigger_2, y_val_tigger))
train_dataset_2 = train_dataset_2.map(preprocess)
train_dataset_2 = train_dataset_2.batch(32)
train_dataset_2 = train_dataset_2.repeat()

# 创建数据集



2024-08-13 09:46:44.253429: W tensorflow/core/framework/cpu_allocator_impl.cc:81] Allocation of 491520000 exceeds 10% of free system memory.


In [19]:
import tensorflow as tf
from tensorflow.keras import layers, models

# 假设 client_model_1, server_model, aux_model 已经被正确定义和初始化
# 定义优化器

optimizer_client = tf.keras.optimizers.Adam()
optimizer_client_1 = tf.keras.optimizers.Adam()
optimizer_server = tf.keras.optimizers.Adam()
loss_fn = tf.keras.losses.CategoricalCrossentropy()


@tf.function
def train_step_client(x, y):
    with tf.GradientTape() as tape_client, tf.GradientTape() as tape_server:
        client_outputs = client_model(x, training=True)
        server_logits = server_model(client_outputs, training=True)
        loss = loss_fn(y, server_logits)
    
    grads_server = tape_server.gradient(loss, server_model.trainable_variables)
    optimizer_server.apply_gradients(zip(grads_server, server_model.trainable_variables))
    
    grads_client = tape_client.gradient(loss, client_model.trainable_variables)
    optimizer_client.apply_gradients(zip(grads_client, client_model.trainable_variables))

    return tf.reduce_mean(loss)

@tf.function
def train_step_combined(x, y):
    with tf.GradientTape() as tape:
        aux_output, server_output = combined_model(x, training=True)
        loss_aux = tf.keras.losses.categorical_crossentropy(y[0], aux_output)
        loss_server = tf.keras.losses.categorical_crossentropy(y[1], server_output)
        loss = 0.5 * loss_aux + 0.5 * loss_server
    
    grads = tape.gradient(loss, combined_model.trainable_variables)
    optimizer_client_1.apply_gradients(zip(grads, combined_model.trainable_variables))
    
    return tf.reduce_mean(loss)

# 冻结 server_model 的所有层
def freeze_server_model():
    for layer in server_model.layers:
        layer.trainable = False

# 取消冻结 server_model 的所有层
def unfreeze_server_model():
    for layer in server_model.layers:
        layer.trainable = True

        
        
        
# 获取 client_model_1 的输出，作为其他两个模型的输入
feature_output = client_model_1.output

# 用 client_model_1 的输出作为输入创建两个任务的模型
server_output = server_model(feature_output)
aux_output = aux_model(feature_output)

combined_model = models.Model(inputs=client_model_1.input, outputs=[aux_output, server_output])
combined_model.compile(optimizer='adam',
                       loss=['categorical_crossentropy', 'categorical_crossentropy'],
                       metrics=['accuracy'])


In [20]:
epochs = 15
steps_per_epoch = len(x_train) // 32

for epoch in range(epochs):
    # 先训练第一部分模型
    unfreeze_server_model()  # 取消冻结 server_model
    for x_batch, y_batch in train_dataset:
        loss_client = train_step_client(x_batch, y_batch)
    print(f"Epoch {epoch + 1}, Client Model Loss: {loss_client.numpy():.4f}")

    # 冻结 server_model 并训练组合模型
    freeze_server_model()  # 冻结 server_model
    for step, (x_batch, (y_batch_aux, y_batch_server)) in enumerate(train_dataset_2.take(steps_per_epoch)):
        loss_combined = train_step_combined(x_batch, (y_batch_aux, y_batch_server))

    unfreeze_server_model()  # 解冻 server_model
    print(f"Epoch {epoch + 1} completed.")

2024-08-13 09:46:47.863354: W tensorflow/core/framework/cpu_allocator_impl.cc:81] Allocation of 491520000 exceeds 10% of free system memory.
2024-08-13 09:47:03.059394: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcublas.so.10
2024-08-13 09:47:07.084112: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudnn.so.7
2024-08-13 09:47:29.324114: W tensorflow/stream_executor/gpu/asm_compiler.cc:81] Running ptxas --version returned 256
2024-08-13 09:47:29.575265: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] Internal: ptxas exited with non-zero error code 256, output: 
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.


Epoch 1, Client Model Loss: 0.8780
Epoch 1 completed.


2024-08-13 10:02:42.285601: W tensorflow/core/framework/cpu_allocator_impl.cc:81] Allocation of 491520000 exceeds 10% of free system memory.
2024-08-13 10:03:11.796981: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:172] Filling up shuffle buffer (this may take a while): 0 of 40000
2024-08-13 10:03:17.053390: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:172] Filling up shuffle buffer (this may take a while): 28271 of 40000
2024-08-13 10:03:19.145932: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:221] Shuffle buffer filled.


Epoch 2, Client Model Loss: 0.6170
Epoch 2 completed.


2024-08-13 10:04:01.986803: W tensorflow/core/framework/cpu_allocator_impl.cc:81] Allocation of 491520000 exceeds 10% of free system memory.


Epoch 3, Client Model Loss: 0.8620
Epoch 3 completed.


2024-08-13 10:04:51.458753: W tensorflow/core/framework/cpu_allocator_impl.cc:81] Allocation of 491520000 exceeds 10% of free system memory.


Epoch 4, Client Model Loss: 0.5941
Epoch 4 completed.
Epoch 5, Client Model Loss: 0.4887
Epoch 5 completed.
Epoch 6, Client Model Loss: 0.4237
Epoch 6 completed.
Epoch 7, Client Model Loss: 0.3177
Epoch 7 completed.
Epoch 8, Client Model Loss: 0.4315
Epoch 8 completed.
Epoch 9, Client Model Loss: 0.2668
Epoch 9 completed.
Epoch 10, Client Model Loss: 0.4441
Epoch 10 completed.
Epoch 11, Client Model Loss: 0.0493
Epoch 11 completed.
Epoch 12, Client Model Loss: 0.1557
Epoch 12 completed.
Epoch 13, Client Model Loss: 0.0500
Epoch 13 completed.
Epoch 14, Client Model Loss: 0.0213
Epoch 14 completed.
Epoch 15, Client Model Loss: 0.0280
Epoch 15 completed.


In [21]:
@tf.function
def train_step_combined_2(x, y):
    with tf.GradientTape() as tape:
        aux_output, server_output = combined_model(x, training=True)
        loss_aux = tf.keras.losses.categorical_crossentropy(y[0], aux_output)
        loss_server = tf.keras.losses.categorical_crossentropy(y[1], server_output)
        loss = 0.5 * loss_aux + 0.5 * loss_server
    
    grads = tape.gradient(loss, combined_model.trainable_variables)
    optimizer_fine_tune.apply_gradients(zip(grads, combined_model.trainable_variables))
    
    return tf.reduce_mean(loss)

In [22]:
###可以看到训练精度还是蛮高的，但是测试精度不理想。过拟合了

In [23]:
from tensorflow.keras.models import Model
def create_combined_model(client_model, server_model, compile_only=False):
    # Getting the input of the client model
    client_input = client_model.input
    
    # Getting the intermediate output by passing the input through the client model
    client_output = client_model(client_input)
    
    # The client model's output is used as the input for the server model
    server_output = server_model(client_output)
    
    # Defining a new model that chains the client and server models
    combined_model = Model(inputs=client_input, outputs=server_output)
    
    # Compile the combined model
    combined_model.compile(optimizer=Adam(),
                           loss='categorical_crossentropy',
                           metrics=['accuracy'])
    
    if not compile_only:
        # If not compile_only, evaluate the model
        loss, accuracy = combined_model.evaluate(x_test, y_test, verbose=0)
        print(f"Test loss: {loss}, Test accuracy: {accuracy}")
    
    return combined_model
combined_model_mian = create_combined_model(client_model_1, server_model, compile_only=True)
combined_model_aux = create_combined_model(client_model_1, aux_model, compile_only=True)

combined_model_client = create_combined_model(client_model, server_model, compile_only=True)


In [24]:
loss, accuracy = combined_model_mian.evaluate(x_val_tigger, y_val_tigger, verbose=1)
##数据集太小会导致过拟合，因此 - loss: 1.7600e-07 - accuracy: 1.0000

313/313 [==============================] - 2s 7ms/step - loss: 0.3086 - accuracy: 0.8923


In [25]:
loss, accuracy = combined_model_aux.evaluate(x_val_tigger, y_val_tigger_2, verbose=1)
#0.9证明全部归为了1。 数据集不均衡，会导致二分类器没办法使用，这里也是一个bug

313/313 [==============================] - 2s 7ms/step - loss: 3.7046e-04 - accuracy: 0.9999


In [26]:
loss, accuracy = combined_model_mian.evaluate(x_test, y_test, verbose=1)
##因为数据集比较小，因此精度会比客户端小很多，正常

313/313 [==============================] - 2s 7ms/step - loss: 0.7015 - accuracy: 0.7873


In [27]:
loss, accuracy = combined_model_mian.evaluate(x_test_tigger, y_test_tigger, verbose=1) 
##后门成功率很高。8221，微调后

313/313 [==============================] - 2s 6ms/step - loss: 0.2470 - accuracy: 0.9490


In [28]:
loss, accuracy = combined_model_client.evaluate(x_test, y_test, verbose=1) 
##后门成功率很高。8221，微调后

313/313 [==============================] - 2s 7ms/step - loss: 0.6376 - accuracy: 0.8297


In [29]:
loss, accuracy = combined_model_client.evaluate(x_test_tigger, y_test_tigger, verbose=1) 
##后门成功率很高。8221，微调后

313/313 [==============================] - 2s 7ms/step - loss: 13.4839 - accuracy: 0.0926


In [30]:

# 假设你有一个已训练的模型
client_model_1.save('my_client_model_1.h5')
server_model.save('my_server_modell.h5')
client_model.save('my_client_model.h5')

In [31]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam
import numpy as np
from tensorflow.keras import layers, models, optimizers, losses
# 冻结 client_model_1 的参数
# 定义优化器
# 冻结 client_model_1 的参数
# 冻结 client_model_1 的参数
client_model_1.trainable = False

# 定义优化器
# 示例梯度裁剪
discriminator_optimizer = optimizers.SGD(lr=0.01, momentum=0.9)
generator_optimizer = optimizers.SGD(lr=0.01, momentum=0.9)
optimizer_client_1 = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9)
# 定义损失函数
bce = losses.BinaryCrossentropy(from_logits=False)


# 定义损失函数
def discriminator_loss(Zs, Zc):
    real_output = discrim_model(Zs)
    fake_output = discrim_model(Zc)
    epsilon = 1e-8  # 避免log(0)
    real_loss = -tf.reduce_mean(tf.math.log(1. - real_output + epsilon))
    fake_loss = -tf.reduce_mean(tf.math.log(fake_output + epsilon))
    total_loss = real_loss + fake_loss
    return total_loss

def generator_loss(Zc):
    fake_output = discrim_model(Zc)
    epsilon = 1e-8  # 避免log(0)
    gen_loss = -tf.reduce_mean(tf.math.log(1. - fake_output + epsilon))
    return gen_loss
# Dc 和 Ds 分别是你的两个输入数据集
# 示例数据加载
Dc = x_train[:10000]  # CIFAR-10 数据，已预处理为适合模型输入
Yc = y_train[:10000]  # CIFAR-10 标签，已转换为 one-hot 编码
Ds = x_val_tigger  # CIFAR-10 数据，已预处理为适合模型输入

# 训练循环

@tf.function
@tf.function
def train_step_client(x, y):
    # 仅针对client_model_1的训练步骤
    with tf.GradientTape() as tape_client:
        # client_model_1前向传播
        client_outputs = client_model(x, training=True)
        # 使用server_model进行前向传播但不更新
        server_logits = server_model(client_outputs, training=False)
        # 计算损失
        loss_1 = loss_fn(y, server_logits)

    # 计算并应用client_model_1的梯度
    grads_client = tape_client.gradient(loss_1, client_model.trainable_variables)
    optimizer_client_1.apply_gradients(zip(grads_client, client_model.trainable_variables))

    return loss_1
loss_fn = tf.keras.losses.CategoricalCrossentropy()

In [32]:
from torch.utils.data import DataLoader
from itertools import cycle

In [33]:
def augment(image, label):
    # 随机水平翻转图像
    image = tf.image.random_flip_left_right(image)
    # 随机调整亮度
    image = tf.image.random_brightness(image, max_delta=0.1)
    # 确保图像值仍然在0到1的范围内
    image = tf.clip_by_value(image, 0.0, 1.0)
    return image, label

In [34]:
#前面结果都挺正常，为什么这里出了问题。
#训练前的精度都很高，但是重新拟合一下精度倒低了，但是期刊的精度还可以。

In [35]:


epochs =  15
batch_size = 64
for epoch in range(epochs):
    for batch in range(0, len(Dc), batch_size):
        Dc_batch = Dc[batch:batch+batch_size]
        Ds_batch = Ds[batch:batch+batch_size]      
        # 生成器生成输出
        Zc = client_model(Dc_batch)
        Zs = client_model_1(Ds_batch)
        # 打印模型输出值
        real_output = discrim_model(Zs)
        fake_output = discrim_model(Zc)
        # 鉴别器的损失计算和优化
        with tf.GradientTape() as disc_tape:
            disc_loss = discriminator_loss(Zs, Zc)
        disc_grads = disc_tape.gradient(disc_loss, discrim_model.trainable_variables)
        discriminator_optimizer.apply_gradients(zip(disc_grads, discrim_model.trainable_variables))

        # 生成器的损失计算和优化
        with tf.GradientTape() as gen_tape:
            Zc = client_model(Dc_batch)  # 重新生成 Zc
            gen_loss = generator_loss(Zc)
        
        gen_grads = gen_tape.gradient(gen_loss, client_model.trainable_variables)
        generator_optimizer.apply_gradients(zip(gen_grads, client_model.trainable_variables))
    
    print(f"Batch {batch//batch_size + 1}: Real output: {real_output.numpy().mean()}, Fake output: {fake_output.numpy().mean()}")
    print(f"Epoch {epoch+1}/{epochs}, Discriminator Loss: {disc_loss.numpy()}, Generator Loss: {gen_loss.numpy()}")
    for batch in range(0, len(Dc), batch_size):  ##只修改这部分
        x_batch = Dc[batch:batch+batch_size]
        y_batch = Yc[batch:batch+batch_size]  
        loss_1 = train_step_client(x_batch, y_batch)
    print("Epoch: {}, Batch: {}, Loss: {:.4f}".format(epoch, batch, loss_1.numpy()))
    
    
for epoch in range(1):
    for batch in range(0, len(Dc), batch_size):
        Dc_batch = Dc[batch:batch+batch_size]
        Ds_batch = Ds[batch:batch+batch_size]      
        # 生成器生成输出
        Zc = client_model(Dc_batch)
        Zs = client_model_1(Ds_batch)
        # 打印模型输出值
        real_output = discrim_model(Zs)
        fake_output = discrim_model(Zc)
        # 鉴别器的损失计算和优化
        with tf.GradientTape() as disc_tape:
            disc_loss = discriminator_loss(Zs, Zc)
        disc_grads = disc_tape.gradient(disc_loss, discrim_model.trainable_variables)
        discriminator_optimizer.apply_gradients(zip(disc_grads, discrim_model.trainable_variables))

        # 生成器的损失计算和优化
        with tf.GradientTape() as gen_tape:
            Zc = client_model(Dc_batch)  # 重新生成 Zc
            gen_loss = generator_loss(Zc)
        
        gen_grads = gen_tape.gradient(gen_loss, client_model.trainable_variables)
        generator_optimizer.apply_gradients(zip(gen_grads, client_model.trainable_variables))
        

Batch 157: Real output: 0.0, Fake output: 0.0
Epoch 1/15, Discriminator Loss: 18.42068099975586, Generator Loss: -0.0
Epoch: 0, Batch: 9984, Loss: 0.0471
Batch 157: Real output: 0.0, Fake output: 0.0
Epoch 2/15, Discriminator Loss: 18.42068099975586, Generator Loss: -0.0
Epoch: 1, Batch: 9984, Loss: 0.0262
Batch 157: Real output: 0.0, Fake output: 0.0
Epoch 3/15, Discriminator Loss: 18.42068099975586, Generator Loss: -0.0
Epoch: 2, Batch: 9984, Loss: 0.0217
Batch 157: Real output: 0.0, Fake output: 0.0
Epoch 4/15, Discriminator Loss: 18.42068099975586, Generator Loss: -0.0
Epoch: 3, Batch: 9984, Loss: 0.0188
Batch 157: Real output: 0.0, Fake output: 0.0
Epoch 5/15, Discriminator Loss: 18.42068099975586, Generator Loss: -0.0
Epoch: 4, Batch: 9984, Loss: 0.0167
Batch 157: Real output: 0.0, Fake output: 0.0
Epoch 6/15, Discriminator Loss: 18.42068099975586, Generator Loss: -0.0
Epoch: 5, Batch: 9984, Loss: 0.0166
Batch 157: Real output: 0.0, Fake output: 0.0
Epoch 7/15, Discriminator Loss

In [36]:
combined_model_mian_c = create_combined_model(client_model, server_model, compile_only=True)

In [37]:
loss, accuracy = combined_model_mian_c.evaluate(x_test, y_test, verbose=1)

313/313 [==============================] - 3s 9ms/step - loss: 0.5479 - accuracy: 0.8499


In [38]:
loss, accuracy = combined_model_mian_c.evaluate(x_test_tigger, y_test_tigger, verbose=1)

313/313 [==============================] - 3s 10ms/step - loss: 13.5856 - accuracy: 0.0995


In [39]:
#解决方案，将模型保存起来，然后用另一个程度的代码跑一下结果看看能不能正常运行。

In [40]:
#313/313 [==============================] - 1s 3ms/step - loss: 0.9457 - accuracy: 0.7466
#Test loss: 0.9456651210784912, Test accuracy: 0.7465999722480774


## 